In [1]:
import numpy as np

In [2]:
import scipy.special

In [3]:
class neuralNetwork:

    # инициализировать нейронную сеть
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # задать количество узлов во входном, скрытом и выходном слоях
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # матрицы весовых коэффициентов связей wio и who.
        # весовые коэффициенты связей между узлами i и j 
        # следующего слоя обозначенны как w_i_j:
        # w11 w21
        # w12 w22 etc
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5),
                                    (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5),
                                    (self.onodes, self.hnodes))

        # коэффициент обучения
        self.lr = learningrate

        # использование сигмоиды в качестве функции активации
        self.activation_function = lambda x: scipy.special.expit(x)

        pass

    # тренировка нейронной сети
    def train(self, inputs_list, targets_list):
        # преобразовать список входных значений
        # в двухмерный массив
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T

        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # рассчитать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)

        # рассчитать входящие сигналя для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)

        # ошибки выходного слоя = (целевое значение "-" фактическое значение)
        output_errors = targets - final_outputs
        # ошибки скрытого слоя - это ошибки output_errors,
        # распределенные пропорционально весовым коэффициентам связей
        # и рекомбинированные на скрытых узлах
        hidden_errors = np.dot(self.who.T, output_errors)

        # обновить весовые коэффициенты для связей между
        # скрытым и выходным слоями
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)),
                                     np.transpose(hidden_outputs))

        # обновить весовые коэффициенты для связей между
        # входным и скрытым слоями
        self.wih += self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)),
                                     np.transpose(inputs))

        pass

    # опрос нейронной сети
    def query(self, inputs_list):
        # преобразовать список входных значений
        # в двухмерный массив
        inputs = np.array(inputs_list, ndmin=2).T

        # рассчитать входящие сигналы для скрытого слоя
        hidden_inputs = np.dot(self.wih, inputs)
        # рассситать исходящие сигналы для скрытого слоя
        hidden_outputs = self.activation_function(hidden_inputs)

        # рассчитать входящие сигналы для выходного слоя
        final_inputs = np.dot(self.who, hidden_outputs)
        # рассчитать исходящие сигналы для выходного слоя
        final_outputs = self.activation_function(final_inputs)

        return final_outputs

In [4]:
# количество входных, скрытых и выходных узлов
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

In [5]:
# коэффициент обучения
learning_rate = 0.1

In [6]:
# создать сеть
n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)


In [17]:
# загрузить набор тренировочных данных
training_data_file = open("mnist_train.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [8]:
# тренировка нейронной сети

# переменная epochs указывает на то сколько раз тренировочный
# набор данных используется для тренировки сети
epochs = 8

for e in range(epochs):
    # проходим циклом во всем элементам
    for record in training_data_list:
        # Получить список значений используя символ (',')
        # в качестве разделителя
        all_values = record.split(',')
        # правильный ответ - первое значение
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # создать целевые выходные значения (все равны 0.01 за исключением
        # желаемого маркерного значения, равного 0.99
        targets = np.zeros(output_nodes) + 0.01

        # all_values[0] - целевое маркерное значение для данной записи
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [16]:
# загрузить тестовый набор данных
test_data_file = open("mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()


In [10]:
# тестирование нейронной сети

# журнал оценок работы нейронной сети, первоночально пустной
scorecard = []

In [11]:
# проходим циклом по всем элементам

for record in test_data_list:
    # Получить список значений используя символ (',')
    # в качестве разделителя
    all_values = record.split(',')
    # правильный ответ - первое значение
    correct_label = int(all_values[0])
    print(correct_label, "истинный маркер")
    # маштабировать и сместить входные значения
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # опрос сети
    outputs = n.query(inputs)
    # инедекс наибольшего значения является маркерным значением
    label = np.argmax(outputs)
    print(label, 'ответ сети')
    # присоединить ответ сети к оценке к концу списка
    if (label == correct_label):
        # в случае корректного ответа сети
        # присоединить к списку значение 1
        scorecard.append(1)
    else:
        # в случае неправильного ответа сети
        # присоединить к списку значение 0
        scorecard.append(0)
        pass

7 истинный маркер
7 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
0 истинный маркер
0 ответ сети
4 истинный маркер
4 ответ сети
1 истинный маркер
1 ответ сети
4 истинный маркер
4 ответ сети
9 истинный маркер
9 ответ сети
5 истинный маркер
5 ответ сети
9 истинный маркер
9 ответ сети
0 истинный маркер
0 ответ сети
6 истинный маркер
6 ответ сети
9 истинный маркер
9 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
5 истинный маркер
5 ответ сети
9 истинный маркер
9 ответ сети
7 истинный маркер
7 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ сети
9 истинный маркер
9 ответ сети
6 истинный маркер
6 ответ сети
6 истинный маркер
6 ответ сети
5 истинный маркер
5 ответ сети
4 истинный маркер
4 ответ сети
0 истинный маркер
0 ответ сети
7 истинный маркер
7 ответ сети
4 истинный маркер
4 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
1 истинный маркер
1 ответ сети
3 истинн

2 ответ сети
4 истинный маркер
4 ответ сети
0 истинный маркер
0 ответ сети
2 истинный маркер
2 ответ сети
7 истинный маркер
7 ответ сети
4 истинный маркер
4 ответ сети
3 истинный маркер
3 ответ сети
3 истинный маркер
3 ответ сети
0 истинный маркер
0 ответ сети
0 истинный маркер
0 ответ сети
3 истинный маркер
3 ответ сети
1 истинный маркер
1 ответ сети
9 истинный маркер
9 ответ сети
6 истинный маркер
6 ответ сети
5 истинный маркер
5 ответ сети
2 истинный маркер
2 ответ сети
5 истинный маркер
5 ответ сети
9 истинный маркер
9 ответ сети
2 истинный маркер
7 ответ сети
9 истинный маркер
9 ответ сети
3 истинный маркер
3 ответ сети
0 истинный маркер
8 ответ сети
4 истинный маркер
4 ответ сети
2 истинный маркер
2 ответ сети
0 истинный маркер
0 ответ сети
7 истинный маркер
7 ответ сети
1 истинный маркер
1 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
5 истинный маркер
5 ответ сети
3 истинный маркер
3 ответ сети
3 истинный маркер
3 ответ 

8 ответ сети
4 истинный маркер
4 ответ сети
6 истинный маркер
6 ответ сети
2 истинный маркер
2 ответ сети
6 истинный маркер
6 ответ сети
7 истинный маркер
7 ответ сети
9 истинный маркер
9 ответ сети
3 истинный маркер
3 ответ сети
2 истинный маркер
2 ответ сети
9 истинный маркер
9 ответ сети
8 истинный маркер
8 ответ сети
2 истинный маркер
2 ответ сети
2 истинный маркер
2 ответ сети
9 истинный маркер
9 ответ сети
2 истинный маркер
2 ответ сети
7 истинный маркер
7 ответ сети
3 истинный маркер
3 ответ сети
5 истинный маркер
5 ответ сети
9 истинный маркер
9 ответ сети
1 истинный маркер
1 ответ сети
8 истинный маркер
8 ответ сети
0 истинный маркер
0 ответ сети
2 истинный маркер
2 ответ сети
0 истинный маркер
0 ответ сети
5 истинный маркер
5 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
7 истинный маркер
7 ответ сети
6 истинный маркер
6 ответ сети
7 истинный маркер
7 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ 

4 ответ сети
4 истинный маркер
4 ответ сети
6 истинный маркер
6 ответ сети
0 истинный маркер
0 ответ сети
2 истинный маркер
2 ответ сети
9 истинный маркер
9 ответ сети
1 истинный маркер
1 ответ сети
4 истинный маркер
4 ответ сети
7 истинный маркер
7 ответ сети
4 истинный маркер
4 ответ сети
7 истинный маркер
7 ответ сети
3 истинный маркер
3 ответ сети
9 истинный маркер
9 ответ сети
8 истинный маркер
8 ответ сети
8 истинный маркер
8 ответ сети
4 истинный маркер
4 ответ сети
7 истинный маркер
7 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
2 истинный маркер
7 ответ сети
3 истинный маркер
3 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
9 истинный маркер
9 ответ сети
1 истинный маркер
1 ответ сети
7 истинный маркер
7 ответ сети
4 истинный маркер
4 ответ сети
0 истинный маркер
0 ответ сети
3 истинный маркер
3 ответ 

2 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
5 истинный маркер
5 ответ сети
0 истинный маркер
0 ответ сети
9 истинный маркер
9 ответ сети
1 истинный маркер
1 ответ сети
7 истинный маркер
9 ответ сети
0 истинный маркер
0 ответ сети
6 истинный маркер
6 ответ сети
0 истинный маркер
0 ответ сети
8 истинный маркер
8 ответ сети
6 истинный маркер
6 ответ сети
8 истинный маркер
8 ответ сети
1 истинный маркер
1 ответ сети
8 истинный маркер
8 ответ сети
0 истинный маркер
0 ответ сети
3 истинный маркер
3 ответ сети
3 истинный маркер
3 ответ сети
7 истинный маркер
7 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
6 истинный маркер
6 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
6 истинный маркер
6 ответ сети
1 истинный маркер
1 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
7 истинный маркер
7 ответ сети
9 истинный маркер
9 ответ сети
0 истинный маркер
0 ответ сети
8 истинный маркер
8 ответ 

...

9 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ сети
5 истинный маркер
5 ответ сети
6 истинный маркер
6 ответ сети
7 истинный маркер
7 ответ сети
8 истинный маркер
8 ответ сети
9 истинный маркер
9 ответ сети
6 истинный маркер
6 ответ сети
9 истинный маркер
9 ответ сети
7 истинный маркер
7 ответ сети
0 истинный маркер
0 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ сети
3 истинный маркер
3 ответ сети
8 истинный маркер
8 ответ сети
5 истинный маркер
5 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
2 истинный маркер
2 ответ сети
0 истинный маркер
0 ответ сети
7 истинный маркер
7 ответ сети
2 истинный маркер
2 ответ сети
6 истинный маркер
6 ответ 

9 ответ сети
4 истинный маркер
4 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ сети
5 истинный маркер
5 ответ сети
6 истинный маркер
6 ответ сети
7 истинный маркер
7 ответ сети
8 истинный маркер
8 ответ сети
9 истинный маркер
9 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ сети
5 истинный маркер
5 ответ сети
6 истинный маркер
6 ответ сети
7 истинный маркер
7 ответ сети
8 истинный маркер
8 ответ сети
9 истинный маркер
9 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ сети
5 истинный маркер
5 ответ сети
6 истинный маркер
6 ответ сети
7 истинный маркер
7 ответ сети
8 истинный маркер
8 ответ сети
9 истинный маркер
9 ответ сети
6 истинный маркер
6 ответ 

0 ответ сети
2 истинный маркер
2 ответ сети
8 истинный маркер
8 ответ сети
0 истинный маркер
0 ответ сети
6 истинный маркер
6 ответ сети
8 истинный маркер
8 ответ сети
1 истинный маркер
1 ответ сети
7 истинный маркер
7 ответ сети
9 истинный маркер
9 ответ сети
1 истинный маркер
1 ответ сети
9 истинный маркер
9 ответ сети
2 истинный маркер
2 ответ сети
6 истинный маркер
6 ответ сети
7 истинный маркер
7 ответ сети
6 истинный маркер
6 ответ сети
6 истинный маркер
6 ответ сети
8 истинный маркер
8 ответ сети
7 истинный маркер
7 ответ сети
4 истинный маркер
4 ответ сети
9 истинный маркер
9 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
3 истинный маркер
3 ответ сети
0 истинный маркер
0 ответ сети
5 истинный маркер
5 ответ сети
5 истинный маркер
5 ответ сети
8 истинный маркер
8 ответ сети
0 истинный маркер
0 ответ сети
3 истинный маркер
3 ответ сети
7 истинный маркер
7 ответ сети
9 истинный маркер
9 ответ сети
7 истинный маркер
7 ответ 

3 ответ сети
6 истинный маркер
6 ответ сети
8 истинный маркер
8 ответ сети
7 истинный маркер
7 ответ сети
1 истинный маркер
1 ответ сети
5 истинный маркер
5 ответ сети
9 истинный маркер
9 ответ сети
9 истинный маркер
9 ответ сети
3 истинный маркер
3 ответ сети
7 истинный маркер
7 ответ сети
2 истинный маркер
2 ответ сети
4 истинный маркер
4 ответ сети
9 истинный маркер
9 ответ сети
4 истинный маркер
4 ответ сети
3 истинный маркер
3 ответ сети
6 истинный маркер
6 ответ сети
2 истинный маркер
2 ответ сети
2 истинный маркер
2 ответ сети
5 истинный маркер
5 ответ сети
3 истинный маркер
3 ответ сети
2 истинный маркер
2 ответ сети
5 истинный маркер
5 ответ сети
5 истинный маркер
5 ответ сети
9 истинный маркер
9 ответ сети
4 истинный маркер
4 ответ сети
1 истинный маркер
1 ответ сети
7 истинный маркер
7 ответ сети
2 истинный маркер
2 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ 

7 ответ сети
6 истинный маркер
6 ответ сети
4 истинный маркер
4 ответ сети
9 истинный маркер
9 ответ сети
9 истинный маркер
9 ответ сети
5 истинный маркер
5 ответ сети
3 истинный маркер
3 ответ сети
7 истинный маркер
7 ответ сети
4 истинный маркер
4 ответ сети
3 истинный маркер
3 ответ сети
0 истинный маркер
0 ответ сети
4 истинный маркер
4 ответ сети
6 истинный маркер
6 ответ сети
6 истинный маркер
6 ответ сети
1 истинный маркер
1 ответ сети
1 истинный маркер
1 ответ сети
3 истинный маркер
3 ответ сети
2 истинный маркер
2 ответ сети
1 истинный маркер
1 ответ сети
0 истинный маркер
0 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ сети
7 истинный маркер
7 ответ сети
8 истинный маркер
8 ответ сети
9 истинный маркер
7 ответ сети
0 истинный маркер
0 ответ сети
1 истинный маркер
1 ответ сети
2 истинный маркер
2 ответ сети
3 истинный маркер
3 ответ сети
4 истинный маркер
4 ответ 

In [13]:
# рассчитать показатель эффективности в виде доли правильных ответов
scorecard_array = np.asarray(scorecard)
print(f"Эффективность = {scorecard_array.sum() / scorecard_array.size * 100} %")

Эффективность = 97.41 %
